*please execute the cell below before starting the tutorial by selecting the cell and pressing Ctrl+Enter*

In [4]:
from aiida.orm import Dict, load_node
from aiida.engine import submit
from aiida import load_profile

load_profile()

# Fleur code

Fleur code is represented by a `FleurCalculation` class in AiiDA.

Generally one needs only two inputs for a Fleur calculation: it is an `inp.xml` file and initial charge density. As you already know, there is a `FleurinpData` class representing an `inp.xml` file. Therefore, there is a `FleurinpData` object in the inputs. 

No class represents charge density in AiiDA-FLEUR and one can not pass in explicitly into a `FleurCalculation`. However, it can be done specifying `RemoteData` object that represent a calculation folder of a previous FLEUR calculation. If a `RemoteData` input is given, the calculation automatically finds previous charge density (and `inp.xml` in some cases) and uses it as an input. This will be explained more in next sub-section.

## Inputs of the Fleur calculation

|name|type|describtion| required |
|:---:|:---:|:---------:|:---:|
|code | Code | Fleur code| yes |
|fleurinpdata | FleurinpData | Object representing inp.xml| no |
|parent_folder | RemoteData | Remote folder of another calculation| no |
|settings | Dict | special settings | no |

**code** is a code object representing Fleur code and stored in the database, it is the only required input. However, it is not enough to specify **code** only: one *must* provide one of supported input configurations:

* **code** + **fleurinpdata** : start a calculation from scratch with given `inp.xml`
* **code** + **parent_folder** : start a calculation using `inp.xml` and charge density from the previous calculation
* **code** + **fleurinpdata** + **parent_folder** : start a calculation using `inp.xml` from given `FleurinpData` and charge density from the previous calculation

In this tutorial we are going to use **code** + **fleurinpdata** configuration only.

**settings** is used to modify a list of files to be copied to/from/between calculation folders and to append command options to the executable. We will not cover this in the tutorial.


## Fleur code submission

### Inputs preparation

First of all, we need to import `FleurnCalculation` class.

In [3]:
from aiida_fleur.calculation.fleur import FleurCalculation

and prepare input nodes. Configured and ready-to-use inpgen code object is already stored in your database under the number pk=xxx. Now you can simple load it from the database:

In [ ]:
fleur_code = load_node(xxx)

We are going to use a FleurinpData that you have created in tutorial 4. Please, remind a PK of an Fe monolayer structure and load in via the same command:

In [ ]:
fleurinpdata = load_node(xxx)

We also need to define computational options:

In [ ]:
options = {'resources' : {"num_machines": 1, "num_mpiprocs_per_machine" : 2}, 'withmpi' : True}

### Assembling inputs in a single dictionaty

Similarly to `FleurinpgenCalculation`, we will use `get_builder()` method to assemble all the inputs:

In [ ]:
inputs = FleurCalculation.get_builder()
inputs.code = fleur_code
inputs.fleurinpdata = fleurinpdata
inputs.metadata.options = options

### Job submission

In [ ]:
res = submit(FleurCalculation, **inputs)
print('The PK of submitted job is'.format(res_pk)

## Results analysis

After FLEUR code is submitted, you can check the status of all running processes:

In [6]:
!verdi process list -p 1

PK    Created    Process label    Process State    Process status
----  ---------  ---------------  ---------------  ----------------

Total results: 0

Info: last time an entry changed state: 1h ago (at 14:21:40 on 2019-09-01)


You can also check the status of inpgen calculation directly. Try running next cell and examine the output:

In [ ]:
!verdi process show 6231

Wait till the process status is 'Finished'. Then let us explore the output nodes. `Fleurcalculation` generates three output nodes:

|name| type | comment|
|:--:|:----:|:----------:|
|output_parameters|Dict| contains parsed `out.xml`|
|remote_folder| FolderData| represents calculation folder |
|retrieved| FolderData | represents retrieved folder |

The main output is `output_parameters` dictionaty containing parsed `out.xml` file. To print it out, run:

In [ ]:
fleur_calc_node = load_node(res_pk)
output_parameters = fleur_calc_node.outputs.output_parameters
print(output_parameters.get_dict())

Or, one can print it out via:

In [7]:
#you need to modify it
!verdi data dict show xxx

Usage: verdi data dict show [OPTIONS] [DATA]...

Error: Invalid value for "[DATA]...": no Dict found with LABEL<xxx>: No result was found


Analysing output dictionary, can you find what is the band gap and total energy of the ground state?

# Scripting tasks

#### 1.  Run a  Fleur calculation for the Si (remote_folder input configuration)

Create a script and run it in the terminal, passing to `FleurCalculation` **code** + **fleurinpdata** + **remote_folder** from a previous calculation and answer the questions: 